Código para la regresión a partir de un proceso gaussiano. Tiene integrados los 
diferentes kernels que se pueden utilizar para llevarla a cabo.

In [ ]:
import numpy as np
import scipy.io
from numpy.random import randn, multivariate_normal
from scipy.linalg import cho_solve, cho_factor
from numpy.linalg import cholesky, solve
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [157]:
dark = np.array([51.0, 51.0, 51.0]) / 255.0
red = np.array([141.0, 45.0, 57.0]) / 255.0
gold = np.array([174.0, 159.0, 109.0]) / 255.0
gray = np.array([175.0, 179.0, 183.0]) / 255.0
lred = np.array([1, 1, 1]) - 0.5 * (np.array([1, 1, 1]) - red)
lgold = np.array([1, 1, 1]) - 0.5 * (np.array([1, 1, 1]) - gold)

In [158]:
def phi(a):
    ell = 1.0
    return 3 * np.exp(-((a - np.linspace(-8, 8, 16).T) ** 2) / (ell ** 2) / 2.0)


F = len(phi(0))  
mu = np.zeros((F, 1))
Sigma = np.eye(F)  

In [159]:
n = 400  
x = np.linspace(-8, 8, n)[:, np.newaxis] 
m = phi(x) @ mu
kxx = phi(x) @ Sigma @ phi(x).T  
s = multivariate_normal(m.flatten(), kxx, size=5).T
stdpi = np.sqrt(np.diag(kxx))[:, np.newaxis]  

X = np.array([-5,-3.4,-2,-1.4,3,4,5.6]).reshape(7,1)
Y = np.array([-2,3.4,0.3,1.4,-3,1,5]).reshape(7,1)

sigma=0
N = len(X)  

M = phi(X) @ mu
kXX = phi(X) @ Sigma @ phi(X).T  
G = kXX + sigma ** 2 * np.eye(N)

G = cho_factor(G)
kxX = phi(x) @ Sigma @ phi(X).T  
A = cho_solve(G, kxX.T).T  

mpost = m + A @ (Y - M)  
vpost = kxx - A @ kxX.T  
spost = multivariate_normal(mpost.flatten(), vpost, size=5).T 
stdpo = np.sqrt(np.diag(vpost))[:, np.newaxis]  

In [149]:
def m(x):
    return phi(x) @ mu


def k(a, b):
    return phi(a) @ Sigma @ phi(b).T

In [162]:
def GP_reg(m, k):
    out = {}
    n = 400  
    x = np.linspace(-8, 8, n)[:, np.newaxis]
    out["mx"] = m(x)
    out["kxx"] = k(x, x)  
    out["s"] = multivariate_normal(m(x).flatten(), k(x, x), size=5).T
    out["stdpi"] = np.sqrt(np.diag(k(x, x)))[:, np.newaxis]

    
    X = np.array([-5,-3.4,-2,-1.4,3,4,5.6]).reshape(7,1)
    Y = np.array([-2,3.4,0.3,1.4,-3,1,5]).reshape(7,1)

    sigma=0
    N = len(X)  

    mX = m(X)
    kXX = k(X, X)  
    G = kXX + sigma ** 2 * np.eye(N)

    G = cho_factor(G)
    kxX = k(x, X)  
    A = cho_solve(G, kxX.T).T 

    out["mpost"] = m(x) + A @ (Y - mX)  
    out["vpost"] = k(x, x) - A @ kxX.T  
    out["spost"] = multivariate_normal(
        out["mpost"].flatten(), out["vpost"], size=5
    ).T 
    out["stdpo"] = np.sqrt(np.diag(out["vpost"]))[:, np.newaxis]

    return out

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc

plt.rcParams["figure.figsize"] = (25, 10)

rc('text', usetex=False)
rc('font', size=14)
rc('legend', fontsize=13)
rc('text.latex', preamble=r'\usepackage{cmbright}')
rc('text.latex', preamble=r'\usepackage[sc]{mathpazo}')


fig, ax = plt.subplots(1, 2)

rc('font', size=14)
rc('legend', fontsize=13)

def GaussPDFscaled(y, m, s):  # shading
    return np.exp(-0.5 * (y - m.T) ** 2 / (s ** 2).T)


GPout = GP_reg(m, k)

yy = np.linspace(-15, 20, 400).reshape([400, 1])
P = GaussPDFscaled(yy, GPout["mx"], stdpi)

ax[0].imshow(
    P, extent=[-8, 8, -15, 20], aspect="auto", origin="lower", cmap="Blues", alpha=0.6
)
ax[0].plot(x, GPout["s"], ":", color='darkmagenta')  # prior
ax[0].plot(x, GPout["mx"], "-", color='skyblue')
ax[0].plot(x, GPout["mx"] + 2 * GPout["stdpi"], "-", color='skyblue')
ax[0].plot(x, GPout["mx"] - 2 * GPout["stdpi"], "-", color='skyblue')
ax[0].set(xlim=[-8, 8], ylim=[-15, 20], title="")

Ppost = GaussPDFscaled(
    yy, GPout["mpost"], GPout["stdpo"]
)  # shading by local marginal pdf
ax[1].imshow(
    Ppost,
    extent=[-8, 8, -15, 20],
    aspect="auto",
    origin="lower",
    cmap="Blues",
    alpha=0.6,
)
ax[1].errorbar(X, Y, yerr=0, fmt="ok")  # data
ax[1].plot(x, GPout["mpost"], "-", color='skyblue')  # posterior mean
ax[1].plot(
    x, GPout["mpost"] + 2 * GPout["stdpo"], "-", color='skyblue'
)  # upper error bars on f
ax[1].plot(
    x, GPout["mpost"] - 2 * GPout["stdpo"], "-", color='skyblue'
)  # lower error bars on f

ax[1].plot(
    x, GPout["mpost"] + 2 * GPout["stdpo"] + 2 * sigma, "-", color='skyblue'
)  # predictive error bars (on y)
ax[1].plot(x, GPout["mpost"] - 2 * GPout["stdpo"] - 2 * sigma, "-", color='skyblue')

ax[1].plot(x, GPout["spost"], ":", color='darkmagenta')  # samples
ax[1].set(xlim=[-8, 8], ylim=[-12, 12], title="")
ax[1].tick_params(axis='both', which='major', labelsize=17)
ax[1].tick_params(axis='both', which='minor', labelsize=15)

plt.show()
fig.savefig("gp_wiener", dpi=200)

In [160]:
def m(x):
    return 0.0 * x  # mean function


def kernel(f):
    return lambda a, b: np.array(
        [[np.float64(f(a[i], b[j])) for j in range(b.size)]
         for i in range(a.size)]
    )

In [73]:
def SE(a, b, ell):
    return 3 ** 2 * np.exp(-((a - b) ** 2) / 2.0 / ell ** 2)


k = kernel(lambda a, b: SE(a, b, 0.5))
#return 3 ** 2 * np.exp(-((a - b) ** 2) / 2.0 / ell ** 2)

In [161]:
def Wiener(a, b, c):
    return np.minimum(a - c, b - c)


k = kernel(lambda a, b: Wiener(a, b, -9))

In [14]:
def OU(a, b, ell):
    return 3 ** 2 * np.exp(-np.abs(a - b) / ell ** 2)


k = kernel(lambda a, b: OU(a, b, 3))

In [42]:
def spline3(a, b, c, o):
    return c ** 2 * (
        np.minimum(a - o, b - o) ** 3 / 3.0
        + 0.5 * np.abs(a - b) * np.minimum(a - o, b - o) ** 2
    )


k = kernel(lambda a, b: spline3(a, b, 1, -9.0))

In [48]:
def linear(a, b):
    return a * b


k = kernel(linear)

In [44]:
def phi(a):
    return ((a + 8.1) / 4) ** 2


k = kernel(lambda a, b: SE(phi(a), phi(b), 1))

In [49]:
k = kernel(lambda a, b: linear(a, b) + SE(a, b, 1))

In [52]:
k = kernel(lambda a, b: linear(a, b) * SE(a, b, 1))